In [ ]:
import matplotlib.pyplot as plt                                     # ########################################################################################### #
import numpy as np                                                  #                                                                                             #
import ipywidgets as widgets                                        # Ez egy interaktív változat akarna lenni, de nagy értékekkel nem működik mindig rendesen!!!! #
from ipywidgets import interact, interactive, HBox, VBox            #                             (Kisebb értékekkel működik)                                     #
                                                                    # ########################################################################################### #


def dc1(c_tömb, p_tömb):
    c_tömb_szélső = np.clip(c_tömb, 1e-8, 1e8)                      # Ha túlnagy lett a szám Nan vagy Inf-é változott, ezért le kellet kerekíteni
    if c_tömb_szélső[0] < 0 or c_tömb_szélső[1] < 0:                # Valamiért néha még így is overflow keletkezik (- értékke válnak) ezt próbáltam megorvosolni sikertelenül
      return 0

    else:
      dcv1 = p_tömb[2] * np.power(c_tömb_szélső[0], p_tömb[4]) * c_tömb_szélső[1] * p_tömb[1]
      return np.clip(dcv1, 1e-8, 1e8)


def dc2(c_tömb, p_tömb):
    c_tömb_szélső = np.clip(c_tömb, 1e-8, 1e8)
    if c_tömb_szélső[0] < 0 or c_tömb_szélső[2] < 0 or c_tömb_szélső[3] < 0:
      return 0
    
    else:
      dcv2 = p_tömb[3] * np.power(c_tömb_szélső[0], p_tömb[5]) * np.power(c_tömb_szélső[2], p_tömb[6]) * np.power(c_tömb_szélső[3], p_tömb[7]) * p_tömb[1]
      return np.clip(dcv2, 1e-8, 1e8)

gráf = widgets.Output()

def szimuláció(ca, cb, cc, cd, ce, cf, nmax, dt, k1, k2, x1, x2, y, z):         # Hogy interaktív lehessen, az egész szimulációnak függvénynek kell lennie amit a 'gráf' fog kiírni
    with gráf:
        gráf.clear_output(wait=True)
        c_tömb = np.array([ca, cb, cc, cd, ce, cf], dtype=np.float64)
        p_tömb = np.array([nmax, dt, k1, k2, x1, x2, y, z])

        ca_oszlop = np.zeros(int(p_tömb[0]) + 1, dtype= np.float64)
        cb_oszlop = np.zeros(int(p_tömb[0]) + 1, dtype= np.float64)
        cc_oszlop = np.zeros(int(p_tömb[0]) + 1, dtype= np.float64)
        cd_oszlop = np.zeros(int(p_tömb[0]) + 1, dtype= np.float64)
        ce_oszlop = np.zeros(int(p_tömb[0]) + 1, dtype= np.float64)
        cf_oszlop = np.zeros(int(p_tömb[0]) + 1, dtype= np.float64)
        t_sor = np.zeros(int(p_tömb[0]) + 1)

        ca_oszlop[0] = c_tömb[0]
        cb_oszlop[0] = c_tömb[1]
        cc_oszlop[0] = c_tömb[2]
        cd_oszlop[0] = c_tömb[3]
        ce_oszlop[0] = c_tömb[4]
        cf_oszlop[0] = c_tömb[5]

        for n in range(1, int(p_tömb[0]) + 1):
            temp_dc1 = dc1(c_tömb, p_tömb)
            temp_dc2 = dc2(c_tömb, p_tömb)
            c_tömb[0] += - temp_dc1 - temp_dc2
            c_tömb[1] += - temp_dc1 + temp_dc2
            c_tömb[2] += 2 * temp_dc1 - temp_dc2
            c_tömb[3] += 2 * temp_dc1 - temp_dc2
            c_tömb[4] += 4 * temp_dc1 + 6 * temp_dc2
            c_tömb[5] += 5 * temp_dc2

            ca_oszlop[n] = c_tömb[0]
            cb_oszlop[n] = c_tömb[1]
            cc_oszlop[n] = c_tömb[2]
            cd_oszlop[n] = c_tömb[3]
            ce_oszlop[n] = c_tömb[4]
            cf_oszlop[n] = c_tömb[5]

            t_sor[n] = t_sor[n - 1] + p_tömb[1]

        c_matrix = np.array([ca_oszlop, cb_oszlop, cc_oszlop, cd_oszlop, ce_oszlop, cf_oszlop])

        hossz = len(c_matrix[0])
        print('cₐ = ' + str(c_matrix[0, 0:hossz:10]),
        '\ncᵦ = ' + str(c_matrix[1, 0:hossz:10]),
        '\ncc = ' + str(c_matrix[2, 0:hossz:10]),
        '\ncd = ' + str(c_matrix[3, 0:hossz:10]),
        '\nce = ' + str(c_matrix[4, 0:hossz:10]),
        '\ncf = ' + str(c_matrix[5, 0:hossz:10]),
        '\nt = ' + str(t_sor[0:hossz:10]))

        plt.clf()                                               # Az előző gráf kitörlése
        fig, ax = plt.subplots()
        ax.set(facecolor='#E0E0E0')
        fig.set(facecolor='snow')

        plt.title('Bray–Liebhafsky reakció szimuláció', fontsize=17, weight='bold')
        plt.xlabel('Eltelt idő (s)', fontsize=15)
        plt.ylabel('c értéke', fontsize=15)

        szín = np.array(['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#beaed4', '#ffff99'])
        név = np.array(['A (H₂O₂)', 'B (I₂)', 'C (IO⁻³)', 'D (H⁺)', 'E (H₂O)', 'F (O₂)'])
        átt = 0.7

        for i in range(0, len(c_matrix)):
            plt.plot(t_sor, c_matrix[i], color=szín[i], label=név[i] + ' anyag koncentrációja', alpha=átt)

        plt.ylim(-0.5, np.max(c_matrix[0:3] + 1))
        plt.grid(linestyle=(0, (1, 1)), color='snow')
        plt.legend()
        plt.show()

csúszka_oszlop1 =VBox([                                                                                     # Hogy két oszlopban lehessenek a csúszkák
    widgets.FloatSlider(value=10, min=0, max=10, step=0.1, description='ca', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=10, step=0.1, description='cb', readout_format='.1f'),          # Minden csúszkának megvan adva az alap érték, min, max és lépésel
    widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='cc', readout_format='.1f'),
    widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='cd', readout_format='.1f'),
    widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='ce', readout_format='.1f'),
    widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='cf', readout_format='.1f'),
    widgets.IntSlider(value=1000, min=500, max=5000, step=100, description='nmax', readout_format='.0f')])

csúszka_oszlop2 = VBox([
    widgets.FloatSlider(value=0.0002, min=0.00001, max=0.001, step=0.00002, description='dt', readout_format='.5f'),
    widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='k1', readout_format='.1f'),
    widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='k2', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=5, step=0.1, description='x1', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=5, step=0.1, description='x2', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=5, step=0.1, description='y', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=5, step=0.1, description='z', readout_format='.1f')])

csúszka_mindoszlop = HBox([csúszka_oszlop1, csúszka_oszlop2])                                           # Összerakom a két oszlopot

widgets.interactive(szimuláció,                                                                         # Emiatt lehet változtatgatni az értékeket
         ca=csúszka_oszlop1.children[0],
         cb=csúszka_oszlop1.children[1],
         cc=csúszka_oszlop1.children[2],                                                                # A szimuláció függvény paramétereihez hozzárendeljük a csúszkákat
         cd=csúszka_oszlop1.children[3],
         ce=csúszka_oszlop1.children[4],
         cf=csúszka_oszlop1.children[5],
         nmax=csúszka_oszlop1.children[6],
         dt=csúszka_oszlop2.children[0],
         k1=csúszka_oszlop2.children[1],
         k2=csúszka_oszlop2.children[2],
         x1=csúszka_oszlop2.children[3],
         x2=csúszka_oszlop2.children[4],
         y=csúszka_oszlop2.children[5],
         z=csúszka_oszlop2.children[6])

elrendezés = VBox([ gráf, csúszka_mindoszlop])                                                          # Felül gráf alul csúszkák

szimuláció(                                                                                             # Szimuláció bekapcsolása az alap értékekkel
    ca=csúszka_oszlop1.children[0].value,
    cb=csúszka_oszlop1.children[1].value,
    cc=csúszka_oszlop1.children[2].value,
    cd=csúszka_oszlop1.children[3].value,
    ce=csúszka_oszlop1.children[4].value,
    cf=csúszka_oszlop1.children[5].value,
    nmax=csúszka_oszlop1.children[6].value,
    dt=csúszka_oszlop2.children[0].value,
    k1=csúszka_oszlop2.children[1].value,
    k2=csúszka_oszlop2.children[2].value,
    x1=csúszka_oszlop2.children[3].value,
    x2=csúszka_oszlop2.children[4].value,
    y=csúszka_oszlop2.children[5].value,
    z=csúszka_oszlop2.children[6].value
)

display(elrendezés)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def dc1(c_tömb, p_tömb, n, rk_tömb):
    dcv1 = p_tömb[2] * np.power(c_tömb[0, n - int(rk_tömb[0])], p_tömb[4]) * c_tömb[1, n - int(rk_tömb[1])] * p_tömb[1]
    return dcv1


def dc2(c_tömb, p_tömb, n, rk_tömb):
    dcv2 = p_tömb[3] * np.power(c_tömb[0, n - int(rk_tömb[0])], p_tömb[5]) * np.power(c_tömb[2, n - int(rk_tömb[2])], p_tömb[6]) * np.power(c_tömb[3, n - int(rk_tömb[3])], p_tömb[7]) * p_tömb[1]
    return dcv2


c_értékek = np.array([10, 2, 0, 0, 0, 0], dtype=np.float64)         #ca, cb, cc, cd, ce, cf
p_tömb = np.array([200, 0.0005, 1, 1, 2, 2, 2, 2, 2, 3, 4, 10])                     #nmax, dt, k1, k2, x1, x2, y, z, nka, nkb, nkc, nkd
rk_tömb = np.zeros((len(p_tömb)- 8))         #rka, rkb, rkc, rkd,
t_sor = np.zeros(int(p_tömb[0]) + 1)

c_matrix = np.zeros((len(c_értékek), int(p_tömb[0]) + 1), dtype=np.float64)

for n in range(0, len(c_értékek)):

    c_matrix[n, 0] = c_értékek[n]

for n in range(1, int(p_tömb[0]) + 1):

    for i in range(0, len(p_tömb) - 8):
        rk_tömb[i] = p_tömb[i + 8]

        if n - rk_tömb[i] < 0:
            rk_tömb[i] = p_tömb[i + 8] - abs(n - rk_tömb[i])


    temp_dc1 = dc1(c_matrix, p_tömb, n, rk_tömb)
    temp_dc2 = dc2(c_matrix, p_tömb, n, rk_tömb)
    c_matrix[0, n] = c_matrix[0, n -1] - temp_dc1 - temp_dc2
    c_matrix[1, n] = c_matrix[1, n -1] - temp_dc1 + temp_dc2
    c_matrix[2, n] = c_matrix[2, n -1] + 2 * temp_dc1 - temp_dc2
    c_matrix[3, n] = c_matrix[3, n -1] + 2 * temp_dc1 - temp_dc2
    c_matrix[4, n] = c_matrix[4, n -1] + 4 * temp_dc1 + 6 * temp_dc2
    c_matrix[5, n] = c_matrix[5, n -1] + 5 * temp_dc2

    t_sor[n] = t_sor[n - 1] + p_tömb[1]



hossz = len(c_matrix[0])
print('cₐ = ' + str(c_matrix[0, 0:hossz:10]),
      '\ncᵦ = ' + str(c_matrix[1, 0:hossz:10]),
      '\ncc = ' + str(c_matrix[2, 0:hossz:10]),
      '\ncd = ' + str(c_matrix[3, 0:hossz:10]),
      '\nce = ' + str(c_matrix[4, 0:hossz:10]),
      '\ncf = ' + str(c_matrix[5, 0:hossz:10]),
      '\nt = ' + str(t_sor[0:hossz:10]))

fig, ax = plt.subplots()
ax.set(facecolor='#E0E0E0')
fig.set(facecolor='snow')

plt.title('Bray–Liebhafsky reakció szimuláció', fontsize=17, weight='bold')
plt.xlabel('Eltelt idő (s)', fontsize=15)
plt.ylabel('c értéke', fontsize=15)

szín = np.array(['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#beaed4', '#ffff99'])
név = np.array(['A (H₂O₂)', 'B (I₂)', 'C (IO⁻³)', 'D (H⁺)', 'E (H₂O)', 'F (O₂)'])
átt = 0.7

for i in range(0, len(c_matrix)- 2):
    plt.plot(t_sor, c_matrix[i], color=szín[i], label=név[i] + ' anyag koncentrációja', alpha=átt)

plt.ylim(-0.5, np.max(c_matrix[0:3] + 1))
plt.grid(linestyle=(0, (1, 1)), color='snow')
plt.legend()
plt.show()

[[10.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets                                        
from ipywidgets import interact, interactive, HBox, VBox


def dc1(c_tömb, p_tömb, n, rk_tömb):
    dcv1 = p_tömb[2] * np.power(c_tömb[0, n - int(rk_tömb[0])], p_tömb[4]) * c_tömb[1, n - int(rk_tömb[1])] * p_tömb[1]
    return dcv1


def dc2(c_tömb, p_tömb, n, rk_tömb):
    dcv2 = p_tömb[3] * np.power(c_tömb[0, n - int(rk_tömb[0])], p_tömb[5]) * np.power(c_tömb[2, n - int(rk_tömb[2])], p_tömb[6]) * np.power(c_tömb[3, n - int(rk_tömb[3])], p_tömb[7]) * p_tömb[1]
    return dcv2

gráf = widgets.Output()

def szimuláció(ca, cb, cc, cd, nmax, dt, k1, k2, x1, x2, y, z, nka, nkb, nkc, nkd):
    gráf.clear_output(wait=True)
    with gráf:        
        c_tömb = np.array([ca, cb, cc, cd, 0, 0], dtype=np.float64)         #ca 10, cb 2, cc 0, cd 0, ce 0, cf 0
        p_tömb = np.array([nmax, dt, k1, k2, x1, x2, y, z, nka, nkb, nkc, nkd])                     #nmax 500, dt 0.0005, k1 1, k2 1, x1 2, x2 2, y 2, z 2, nka 1, nkb 1, nkc 1, nkd 1
        rk_tömb = np.zeros((len(p_tömb)- 8))         #rka, rkb, rkc, rkd,
        t_sor = np.zeros(int(p_tömb[0]) + 1)

        c_matrix = np.zeros((len(c_tömb), int(p_tömb[0]) + 1), dtype=np.float64)

        for n in range(0, len(c_tömb)):

            c_matrix[n, 0] = c_tömb[n]

        for n in range(1, int(p_tömb[0]) + 1):

            for i in range(0, len(rk_tömb)):
                rk_tömb[i] = p_tömb[i + 8]

                if n - rk_tömb[i] < 0:
                    rk_tömb[i] = p_tömb[i + 8] - abs(n - rk_tömb[i])


            temp_dc1 = dc1(c_matrix, p_tömb, n, rk_tömb)
            temp_dc2 = dc2(c_matrix, p_tömb, n, rk_tömb)
            c_matrix[0, n] = c_matrix[0, n -1] - temp_dc1 - temp_dc2
            c_matrix[1, n] = c_matrix[1, n -1] - temp_dc1 + temp_dc2
            c_matrix[2, n] = c_matrix[2, n -1] + 2 * temp_dc1 - temp_dc2
            c_matrix[3, n] = c_matrix[3, n -1] + 2 * temp_dc1 - temp_dc2
            c_matrix[4, n] = c_matrix[4, n -1] + 4 * temp_dc1 + 6 * temp_dc2
            c_matrix[5, n] = c_matrix[5, n -1] + 5 * temp_dc2

            t_sor[n] = t_sor[n - 1] + p_tömb[1]


        hossz = len(c_matrix[0])
        print('cₐ = ' + str(c_matrix[0, 0:hossz:50]),
            '\ncᵦ = ' + str(c_matrix[1, 0:hossz:50]),
            '\ncc = ' + str(c_matrix[2, 0:hossz:50]),
            '\ncd = ' + str(c_matrix[3, 0:hossz:50]),
            '\nce = ' + str(c_matrix[4, 0:hossz:50]),
            '\ncf = ' + str(c_matrix[5, 0:hossz:50]),
            '\nt = ' + str(t_sor[0:hossz:10]))
        
        plt.clf()                          
        fig, ax = plt.subplots()
        ax.set(facecolor='#E0E0E0')
        fig.set(facecolor='snow')

        plt.title('Bray–Liebhafsky reakció szimuláció', fontsize=17, weight='bold')
        plt.xlabel('Eltelt idő (s)', fontsize=15)
        plt.ylabel('c értéke', fontsize=15)

        szín = np.array(['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#beaed4', '#ffff99'])
        név = np.array(['A (H₂O₂)', 'B (I₂)', 'C (IO⁻³)', 'D (H⁺)', 'E (H₂O)', 'F (O₂)'])
        átt = 0.7

        for i in range(0, len(c_matrix)- 2):
            plt.plot(t_sor, c_matrix[i], color=szín[i], label=név[i] + ' anyag koncentrációja', alpha=átt)

        plt.ylim(-0.5, np.max(c_matrix[0:3] + 1))
        plt.grid(linestyle=(0, (1, 1)), color='snow')
        plt.legend()
        plt.show()

csúszka_oszlop1 =VBox([
    widgets.FloatSlider(value=10, min=0, max=10, step=0.1, description='ca', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=10, step=0.1, description='cb', readout_format='.1f'),         
    widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='cc', readout_format='.1f'),
    widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='cd', readout_format='.1f'),         
    widgets.IntSlider(value=500, min=100, max=5000, step=100, description='nmax', readout_format='.0f'),
    widgets.FloatSlider(value=0.0005, min=0.00005, max=0.001, step=0.00002, description='dt', readout_format='.5f'),
    widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='k1', readout_format='.1f'),
    widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='k2', readout_format='.1f')])

csúszka_oszlop2 = VBox([
    widgets.IntSlider(value=1, min=1, max=10, step=1, description='nka', readout_format='.0f'),
    widgets.IntSlider(value=1, min=1, max=10, step=1, description='nkc', readout_format='.0f'),
    widgets.IntSlider(value=1, min=1, max=10, step=1, description='nkb', readout_format='.0f'),
    widgets.IntSlider(value=1, min=1, max=10, step=1, description='nkd', readout_format='.0f'),
    widgets.FloatSlider(value=2, min=0, max=3, step=0.1, description='x1', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=3, step=0.1, description='x2', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=3, step=0.1, description='y', readout_format='.1f'),
    widgets.FloatSlider(value=2, min=0, max=3, step=0.1, description='z', readout_format='.1f'),
])   


csúszka_mindoszlop = HBox([csúszka_oszlop1, csúszka_oszlop2])                                           

widgets.interact(szimuláció,
    ca=csúszka_oszlop1.children[0],
    cb=csúszka_oszlop1.children[1],
    cc=csúszka_oszlop1.children[2],                                                                # A szimuláció függvény paramétereihez hozzárendeljük a csúszkákat
    cd=csúszka_oszlop1.children[3],
    nmax=csúszka_oszlop1.children[4],
    dt=csúszka_oszlop1.children[5],
    k1=csúszka_oszlop1.children[6],
    k2=csúszka_oszlop1.children[7],
    nka=csúszka_oszlop2.children[0],
    nkb=csúszka_oszlop2.children[1],
    nkc=csúszka_oszlop2.children[2],
    nkd=csúszka_oszlop2.children[3],
    x1=csúszka_oszlop2.children[4],
    x2=csúszka_oszlop2.children[5],
    y=csúszka_oszlop2.children[6],
    z=csúszka_oszlop2.children[7])


elrendezés = VBox([ gráf, csúszka_mindoszlop])  

display(elrendezés)


interactive(children=(FloatSlider(value=10.0, description='ca', max=10.0, readout_format='.1f'), FloatSlider(v…